In [ ]:
import tables as tb
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import astropy.units as u
from astropy.time import Time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import re
import math as mt
from scipy.stats import norm
import scipy
import seaborn as sns
from astropy.table import Table, vstack
import glob
from datetime import datetime
from python_modules.muonpipe import usefull_func
import glob
OPT_EFF=[0.6, 0.7, 0.8, 0.9, 1.0]
MIR_MIS=[0.00125, 0.0025, 0.0035, 0.005, 0.0075, 0.01, 0.0125]
colors = ['C0', 'C1', 'C2', 'C3', 'C4']

size_outside = 500 # phe



### Recent tries of datacheck processing

In [ ]:
files = glob.glob('/Users/vdk/muons2024/v0.9-v0.10/20*/DL1_datacheck_*.h5')
files.sort()

runsummary = []
cosmics = []
for file in files:
    runsummary.append(pd.read_hdf(file, 'runsummary'))
    cosmics.append(pd.read_hdf(file, 'cosmics'))
    
cosmics_pd = pd.concat(cosmics, ignore_index=True)
runsummary_pd = pd.concat(runsummary, ignore_index=True)
cosmics_pd.columns

In [ ]:
runsummary_pd.columns

In [ ]:
runs_with_0_rings = []
# Lets try to check if all subruns have muon rings
for run, num_rings in zip(cosmics_pd['runnumber'], cosmics_pd['num_contained_mu_rings']):
    if num_rings == 0:
        runs_with_0_rings.append(run)
        #print(run)

In [ ]:
runsummary_pd[runsummary_pd['runnumber'].isin([16449])]

In [ ]:
# Lets choose only values with zenith anlge that differs not much than 1 degree
def delta(a,b):
    return abs(np.rad2deg(a)-np.rad2deg(b))

#runsummary_pd['runnumber'][(np.degrees(runsummary_pd['_altitude']) > 44) & (np.degrees(runsummary_pd['mean_altitude']) < 45)]) #& (runsummary_pd['mu_radius_mean'] > 0.95) & (runsummary_pd['mu_radius_mean'] < 1.3)]
zd_runnumbers = runsummary_pd['runnumber'][
    (delta(runsummary_pd['min_altitude'],runsummary_pd['max_altitude'])< 1) & 
    (delta(runsummary_pd['min_azimuth'],runsummary_pd['max_azimuth'])< 1) & 
    (runsummary_pd['mu_effi_stddev']<=0.02) & 
    (runsummary_pd['mu_width_stddev']<0.02) &
    (runsummary_pd['num_contained_mu_rings'])
]
zenith_table = runsummary_pd[runsummary_pd['runnumber'].isin(zd_runnumbers)]

In [ ]:
zenith_table

In [ ]:
plt.figure(figsize = (12,9))
sns.regplot(x = np.degrees(zenith_table['mean_altitude']), y = zenith_table['mu_effi_mean'], color = 'k', label = 'Mean Optical Efficiency during one observation')#, x_bins=75
plt.errorbar(np.degrees(zenith_table['mean_altitude']), zenith_table['mu_effi_mean'], yerr=zenith_table['mu_effi_stddev'], fmt = 'none', capsize=5, zorder=1, color='C0', alpha = 0.4)
plt.grid(alpha = 0.3)
plt.title("Optical Efficiency for all LST1 observational time")
#plt.vlines(3000,0,0.3)
#plt.ylim(0.1,0.275)
# plt.fill_betweenx(y=[0,0.3], x1 = 0, x2=x_fill[1], alpha = 0.05)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[1], x2=x_fill[2], alpha = 0.075)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[2], x2=x_fill[3], alpha = 0.075)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[3], x2=x_fill[4], alpha = 0.075)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[4], x2=x_fill[5], alpha = 0.075)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[5], x2=18000, alpha = 0.075)
#plt.xlim(0,18000)
plt.xlabel('Zenith angle')
plt.ylabel('Mean optical efficiency')
plt.legend()

In [ ]:
plt.figure(figsize = (12,9))
sns.regplot(x = np.degrees(zenith_table['mean_azimuth']), y = zenith_table['mu_effi_mean'], color = 'k', label = 'Mean Optical Efficiency during one observation')#, x_bins=75
plt.errorbar(np.degrees(zenith_table['mean_azimuth']), zenith_table['mu_effi_mean'], yerr=zenith_table['mu_effi_stddev'], fmt = 'none', capsize=5, zorder=1, color='C0', alpha = 0.4)
#plt.errorbar(mean_runs, mean_opt_efficiency, yerr=std_opt, fmt = 'none', capsize=5, zorder=1, color='C0', alpha = 0.4)
plt.grid(alpha = 0.3)
plt.title("Optical Efficiency for all LST1 observational time")
#plt.vlines(3000,0,0.3)
#plt.ylim(0.1,0.275)
# plt.fill_betweenx(y=[0,0.3], x1 = 0, x2=x_fill[1], alpha = 0.05)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[1], x2=x_fill[2], alpha = 0.075)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[2], x2=x_fill[3], alpha = 0.075)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[3], x2=x_fill[4], alpha = 0.075)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[4], x2=x_fill[5], alpha = 0.075)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[5], x2=18000, alpha = 0.075)
#plt.xlim(0,18000)
plt.xlabel('Zenith angle')
plt.ylabel('Mean optical efficiency')
plt.legend()

In [ ]:
plt.figure(figsize = (12,9))
sns.regplot(x = np.degrees(zenith_table['mean_altitude']), y = zenith_table['mu_intensity_mean'], color = 'k', label = 'Mean Optical Efficiency during one observation')#, x_bins=75
#plt.errorbar(np.degrees(zenith_table['mean_altitude']), zenith_table['mu_effi_mean'], yerr=zenith_table['mu_effi_stddev'], fmt = 'none', capsize=5, zorder=1, color='C0', alpha = 0.4)
plt.grid(alpha = 0.3)
plt.title("Optical Efficiency for all LST1 observational time")
#plt.vlines(3000,0,0.3)
#plt.ylim(0.1,0.275)
# plt.fill_betweenx(y=[0,0.3], x1 = 0, x2=x_fill[1], alpha = 0.05)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[1], x2=x_fill[2], alpha = 0.075)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[2], x2=x_fill[3], alpha = 0.075)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[3], x2=x_fill[4], alpha = 0.075)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[4], x2=x_fill[5], alpha = 0.075)
# plt.fill_betweenx(y=[0,0.3], x1 = x_fill[5], x2=18000, alpha = 0.075)
#plt.xlim(0,18000)
plt.xlabel('Zenith angle')
plt.ylabel('Mean optical efficiency')
plt.legend()

In [ ]:
z,x,c = plt.hist(runsummary_pd['mu_effi_stddev'], bins = 50)
plt.xlim(0,0.02)

In [ ]:
z,x,c = plt.hist(np.degrees(zenith_table['mean_altitude']), bins = 40)


In [ ]:
z,x,c = plt.hist(zenith_table['mu_width_mean'], bins = 40)

In [ ]:
z,x,c = plt.hist(runsummary_pd['mu_intensity_mean'],bins = 50)
plt.xlim(0,5000)

In [ ]:
z,x,c = plt.hist(cosmics_pd['mu_radius_mean'],bins = 50)
min(cosmics_pd['mu_radius_mean'])

### Looking for other quality cuts

In [ ]:
runsummary_pd['elapsed_time']

In [ ]:
z,x,c = plt.hist(runsummary_pd['cosmics_fraction_pulses_above30']*runsummary_pd['elapsed_time'], bins = 40)

### Rubens code

In [ ]:
date=20200114

listdir = glob.glob('/Users/vdk/muons2024/20240131muons/muons_LST-1.Run*')
# Muons analyzed using GlobalPeakWindowSum

dat = Table.read('{}'.format(listdir[0]), format='fits')
for i in range(1,len(listdir)):
    dat2 = Table.read('{}'.format(listdir[i]), format='fits')
    dat = vstack([dat, dat2])

df = dat.to_pandas()
df_good_data = df[df['good_ring']]

In [ ]:
def format_axes(ax,option):
    
    if(option=='width'):
        ax.set_xlabel('Muon radius [deg]')
        ax.set_ylabel('Muon ring width [deg]')
        ax.grid(linestyle="dashed")
        #ax.set_ylim(0.042,0.07)
        ax.legend()
    if(option=='size'):
        ax2.grid(linestyle="dashed")
        ax2.legend(ncol=2)
        ax2.set_xlabel('Muon radius [deg]')
        ax2.set_ylabel('Muon Size [pe]')
        ax2.set_ylim(1500,4500)
        
fig,ax = plt.subplots()
fig2,ax2 = plt.subplots()



sns.regplot(x = df_good_data['ring_radius'][(df_good_data['ring_completeness'] > 0.9) & (df_good_data['size_outside'] < size_outside)], 
            y = df_good_data['ring_width'][(df_good_data['ring_completeness'] > 0.9) & (df_good_data['size_outside'] < size_outside)], 
            x_bins=10, ax=ax, color = 'k', label = 'Real Data')
sns.regplot(x = df_good_data['ring_radius'][(df_good_data['ring_completeness'] > 0.9) & (df_good_data['size_outside'] < size_outside)], 
            y = df_good_data['ring_size'][(df_good_data['ring_completeness'] > 0.9) & (df_good_data['size_outside'] < size_outside)], 
            x_bins=10, ax=ax2, color = 'k', label = 'Real Data')

format_axes(ax,'width')        
format_axes(ax2,'size') 

In [ ]:
def Calculate_stereo_rate(RingComp,RINGCOMPCUT,RingCont,RINGCONTCUT,ImpactDiff,IMPACTDIFFCUT):   
    """
    RingComp - completeness
    RingCont - containment
    ImpactDiff - difference in sim and rec impact parameter
    RINGCOMPCUT,RINGCONTCUT,IMPACTDIFFCUT - cuts on the appropriate variables
    """ 
    VIEWANGLE = 3.5 # deg
    IMPACT_PARAMETER = 150 # m
    Flux = 200 # m^-2 s^-1 sr^-1

    Solid_angle = 2 * mt.pi * (1 - mt.cos(np.radians(3.5)))
    Area = mt.pi * mt.pow(IMPACT_PARAMETER,2)
    Nsim = 1e7
    Nrec = len(RingCont[(RingComp > RINGCOMPCUT) & 
                        (RingCont > RINGCONTCUT) & 
                        (ImpactDiff < IMPACTDIFFCUT)])
    print("Nrec = ", Nrec)
    rate = Flux * Solid_angle * Area * Nrec / Nsim  # Hz
    erate = Flux * Solid_angle * Area * mt.sqrt(Nrec) / Nsim # Hz
    print("Number of seconds simulated: ", Nsim / (Flux * Solid_angle * Area) )
    
    return rate, erate

In [ ]:
#maindir='../../../cta-lstchain-extra/muons/simulations/lstchain_v0/Stereo/Array_centered/'

RINGCOMPCUT = 0.9 # containment of pixels above threshold inside a ring
RINGCONTCUT = 0.9 # Angular cut for muon containment within the camera
IMPACTDIFFCUT = 10. # Difference between reconstructed and real Impact parameter

#sfn = sorted(os.listdir(maindir + 'files')) # Sort the filename list

maindir = '/Users/vdk/muons2024/20240131muons/'
sfn = os.listdir('/Users/vdk/muons2024/20240131muons/')

RingWidth = []
MuonEff = []
ImpactParam = []
RingCont = []
RingComp = []
RingPixComp = []
Core_x = []
Core_y = []
Impact_arr_x = []
Impact_arr_y = []
MCImpactP = []
ImpactDiff = []
RingSize = []
RingRadius = []
NTels = []

for filename in sfn:
    if filename.endswith(".fits"):
        #print("Opening file: ", filename)
        table=fits.open("%s/%s" % (maindir,filename))
        data=table[1].data
        
        MuonEff = np.concatenate([MuonEff,data.field(0)])
        RingWidth = np.concatenate([RingWidth,data.field(2)])
        ImpactParam = np.concatenate([ImpactParam,data.field(1)])
        RingCont = np.concatenate([RingCont,data.field(3)])
        RingComp = np.concatenate([RingComp,data.field(4)])
        RingPixComp = np.concatenate([RingPixComp,data.field(5)])
        Core_x = np.concatenate([Core_x,data.field(6)])
        Core_y = np.concatenate([Core_y,data.field(7)])
        Impact_arr_x = np.concatenate([Impact_arr_x,data.field(8)])
        Impact_arr_y = np.concatenate([Impact_arr_y,data.field(9)])
        MCImpactP = np.concatenate([MCImpactP,data.field(10)])
        ImpactDiff = np.concatenate([ImpactDiff,data.field(11)])
        RingSize = np.concatenate([RingSize,data.field(12)])
        RingRadius = np.concatenate([RingRadius,data.field(13)])
        NTels = np.concatenate([NTels,data.field(14)])

rate, erate = Calculate_stereo_rate(RingPixComp, -1, RingCont, -1, ImpactDiff, 1e10)
print("The rate of muons is %.1f+-%.1f Hz" % (rate, erate))

rate, erate = Calculate_stereo_rate(RingPixComp, -1, RingCont, -1, ImpactDiff, IMPACTDIFFCUT)
print("The rate of well reconstructed muons (< %i m) containment is %.1f+-%.1f Hz" % (IMPACTDIFFCUT, rate, erate))

rate, erate = Calculate_stereo_rate(RingPixComp, RINGCOMPCUT, RingCont, -1, ImpactDiff, 1e10)
print("The rate of well reconstructed muons (> %i%%) pixel ring containment is %.1f+-%.1f Hz" % (RINGCOMPCUT*100, rate, erate))

rate, erate = Calculate_stereo_rate(RingComp, RINGCOMPCUT, RingCont, -1, ImpactDiff, 1e10)
print("The rate of well reconstructed muons (> %i%%) ring completeness is %.1f+-%.1f Hz" % (RINGCOMPCUT*100, rate, erate))

rate, erate = Calculate_stereo_rate(RingPixComp, -1, RingCont, RINGCONTCUT, ImpactDiff, 1e10)
print("The rate of well reconstructed muons (> %i%%) ring containment is %.1f+-%.1f Hz" % (RINGCONTCUT*100, rate, erate))

rate, erate = Calculate_stereo_rate(RingPixComp, RINGCOMPCUT, RingCont, RINGCONTCUT, ImpactDiff, IMPACTDIFFCUT)
print("The rate of well reconstructed muons all cuts is %.1f+-%.1f Hz" % (rate, erate))

### Datacheck hdf5 processing

In [ ]:
# Path to your HDF5 file
dl1_file = '/Users/vdk/muons2024/DL1_datacheck_20240131.h5'

fi = h5py.File(dl1_file, 'r')
keys = list(fi.keys())

def printname(name):
    print(name)

# Show all included tables
fi.visit(printname)
pd_hdf = pd.read_hdf(dl1_file, 'runsummary/table')
pd_hdf_cosmics = pd.read_hdf(dl1_file, 'cosmics/table')
pd_hdf.columns


In [ ]:
for column in pd_hdf.columns:
    print(column)

In [ ]:
for column in pd_hdf_cosmics.columns:
    print(column)

In [ ]:
for item in pd_hdf_cosmics['mu_intensity_mean']:

In [ ]:
for item in fi['runsummary/table']:
    print(item)

### Lets choose only runs where change in telescope pointing direction does not exceed 1 deg

In [ ]:

# Specify the directory you want to start from
#root_dir = '/Users/vdk/muons2024/datacheck_v0.10/'
root_dir = '/Users/vdk/muons2024/v0.9'

# Pattern to match all items in root_dir that are directories
pattern = os.path.join(root_dir, '*/')
run_numbers= {}

for dir_path in glob.glob(pattern):
    if dir_path[-5:-3] in ['11','12', '01', '02','03', '04']:
        dl1_file = dir_path+f'DL1_datacheck_{dir_path[-9:-1]}.h5'
        print(f"reading {dl1_file}")
        pd_hdf = pd.read_hdf(dl1_file, 'runsummary/table')
        for i,item in enumerate(pd_hdf['mean_altitude']):
            if (np.rad2deg(abs(pd_hdf['min_altitude'][i] - pd_hdf['max_altitude'][i])) <= 1) and \
               (np.rad2deg(abs(pd_hdf['min_azimuth'][i] - pd_hdf['max_azimuth'][i])) <= 1):
                run_numbers[pd_hdf['runnumber'][i]] = f"{np.rad2deg(pd_hdf['mean_altitude'][i])},{np.rad2deg(pd_hdf['mean_azimuth'][i])}"
        #print(dir_path+f'DL1_datacheck_{dir_path[-9:-1]}.h5')


In [ ]:
print(f"There are {len(run_numbers)} winter runs with small changes in telescope direction ")
altitudes = []
azimuthes = []

for key,item in run_numbers.items():
   #print(round(float(item.split(',')[0]),2),round(float(item.split(',')[1]),2))
   altitudes.append(round(float(item.split(',')[0]),2))
   azimuthes.append(round(float(item.split(',')[1]),2))

runs = [i for i in range(len(altitudes))]

fig, axs = plt.subplots(1, 2, figsize=(12, 5))#, sharey=True)
axs[0].scatter(runs,altitudes, s = 20, marker = 'x')
axs
axs[0].grid(alpha = 0.5)
axs[1].scatter(runs,azimuthes, s = 20, marker = 'x')
axs[1].grid(alpha = 0.5)
axs[0].set_xlabel('Run')
axs[1].set_xlabel('Run')
axs[0].set_ylabel('Degree')
axs[0].set_title('Telescope Altitudes')
axs[1].set_title('Telescope Azimuthes')
#plt.scatter(runs,altitudes)
#plt.scatter(runs,azimuthes)



### Check time of each observation

In [ ]:
timestamp = 1.70673312e+09
dt_object = datetime.utcfromtimestamp(timestamp)

# Format the datetime object to a string in a readable format
formatted_time = dt_object.strftime('%Y-%m-%d %H:%M:%S')

formatted_time
for time in pd_hdf['time']:
    dt_object = datetime.utcfromtimestamp(time)
    formatted_time = dt_object.strftime('%Y-%m-%d %H:%M:%S')
    print(formatted_time)


In [ ]:
altitudes = []
for i,item in enumerate(pd_hdf['mean_altitude']):
    altitudes.append(np.rad2deg(item))
    #print(f"for run number {pd_hdf['runnumber'][i]} the difference in altitude during run is {np.rad2deg(abs(pd_hdf['min_altitude'][i] - pd_hdf['max_altitude'][i]))} degrees")

azimuthes = []
for i,item in enumerate(pd_hdf['mean_altitude']):
    azimuthes.append(np.rad2deg(item))
    #print(f"for run number {pd_hdf['runnumber'][i]} the difference in altitude during run is {np.rad2deg(abs(pd_hdf['min_azimuth'][i] - pd_hdf['max_azimuth'][i]))} degrees")

In [ ]:
log_file_path = '/Users/vdk/muons2024/DL1_datacheck_20240131.log'

with open(log_file_path, 'r') as file:
    print_next_line = False
    for line in file:
        if print_next_line:
            print(line.strip())  # Print the next line
            print_next_line = False
        elif line.startswith('Anomalies'):
            print(line.strip())
            print_next_line = True
        

In [ ]:
log_file_path = '/Users/vdk/muons2024/DL1_datacheck_20240131.log'

with open(log_file_path, 'r') as file:
    for line in file:
        print(line.strip())

In [ ]:


log_file_path = '/Users/vdk/muons2024/DL1_datacheck_20240109.log'

with open(log_file_path, 'r') as file:
    for line in file:
        print(line.strip())

### Perform all quality cuts on datacheck

In [ ]:

# Specify the directory you want to start from
#root_dir = '/Users/vdk/muons2024/datacheck_v0.10/'
root_dir = '/Users/vdk/muons2024/v0.9/'

# Pattern to match all items in root_dir that are directories
pattern = os.path.join(root_dir, '*/')
optical_eff = {}
num_cosmic = {}
elapsed_time = {}
date = {}
mu_width = {}
mu_number = []


for dir_path in glob.glob(pattern):
    # Choose only winter
    #if dir_path[-5:-3] in ['11','12', '01', '02','03', '04']:
    if True:
        dl1_file = dir_path+f'DL1_datacheck_{dir_path[-9:-1]}.h5'
        print(f"reading {dl1_file}")
        pd_hdf = pd.read_hdf(dl1_file, 'runsummary/table')
        pd_hdf_cosmic = pd.read_hdf(dl1_file, 'cosmics/table')
        for i,item in enumerate(pd_hdf['mean_altitude']):
            # Choose only narrow FoV
            # if (np.rad2deg(abs(pd_hdf['min_altitude'][i] - pd_hdf['max_altitude'][i])) <= 1) and \
            #    (np.rad2deg(abs(pd_hdf['min_azimuth'][i] - pd_hdf['max_azimuth'][i])) <= 1):
            if True:
                if pd_hdf['mu_effi_mean'][i] < 0.3:
                    #num_cosmic[int(pd_hdf['runnumber'][i])] = (pd_hdf['num_cosmics'][i])
                    num_cosmic[pd_hdf['runnumber'][i]] = pd_hdf['num_cosmics'][i]
                    elapsed_time[int(pd_hdf['runnumber'][i])] = (pd_hdf['elapsed_time'][i])
                    date[int(pd_hdf['runnumber'][i])] = (pd_hdf['time'][i])
                    optical_eff[int(pd_hdf['runnumber'][i])] = (pd_hdf['mu_effi_mean'][i])
                    mu_width[int(pd_hdf['runnumber'][i])] = (pd_hdf['mu_width_mean'][i])
                    mu_number.append(pd_hdf['num_contained_mu_rings'][i])
        #print(dir_path+f'DL1_datacheck_{dir_path[-9:-1]}.h5')


In [ ]:
num = [item for key,item in num_cosmic.items()]
usefull_func.hist_wo_outliers(num,outlier_value=10000000,bin_number=20,xlabel='Number of cosmics in the run')

In [ ]:
usefull_func.hist_wo_outliers(mu_number,outlier_value=10000,bin_number=20,xlabel='Number of muons in the run')

In [ ]:
sorted_dict = {key: optical_eff[key] for key in sorted(optical_eff)}
runs = [key for key in sorted_dict.keys()]
effs = [sorted_dict[key] for key in sorted_dict.keys()]
plt.scatter(runs, effs, marker = 'x', s = 15, alpha = 0.5)
plt.grid(alpha = 0.5)
plt.xlabel('Run Number')
plt.ylabel('Optical Efficiency')

In [ ]:
len(runs)

In [ ]:
plt.figure(figsize = (12,9))
sns.regplot(x = runs, y = effs, x_bins=3000, color = 'k', label = 'Real Data')
plt.grid(alpha = 0.5)
plt.title("Optical Efficiency")

In [ ]:

# Specify the directory you want to start from
root_dir = '/Users/vdk/muons2024/datacheck_v0.10/'
root_dir = '/Users/vdk/muons2024/v0.9/'

# Pattern to match all items in root_dir that are directories
pattern = os.path.join(root_dir, '*/')
optical_eff = {}
num_cosmic = {}
elapsed_time = {}
date = {}
mu_width = {}
mu_number = []
altitude = {}
azimuth = {}
list_of_dl1_files = []

for dir_path in glob.glob(pattern):
    # Choose only winter
    if dir_path[-5:-3] in ['11','12', '01', '02','03', '04']:
        dl1_file = dir_path+f'DL1_datacheck_{dir_path[-9:-1]}.h5'
        print(f"reading {dl1_file}")
        pd_hdf = pd.read_hdf(dl1_file, 'runsummary/table')
        pd_hdf_cosmic = pd.read_hdf(dl1_file, 'cosmics/table')
        for i,item in enumerate(pd_hdf['mean_altitude']):
            # Choose only narrow FoV
            if (np.rad2deg(abs(pd_hdf['min_altitude'][i] - pd_hdf['max_altitude'][i])) <= 1) and \
               (np.rad2deg(abs(pd_hdf['min_azimuth'][i] - pd_hdf['max_azimuth'][i])) <= 1):
                if pd_hdf['mu_effi_mean'][i] < 0.3:
                    if pd_hdf['num_contained_mu_rings'][i] < 3500 and pd_hdf['num_contained_mu_rings'][i] > 1500: 
                        if pd_hdf['num_cosmics'][i] < 10000000 and pd_hdf['num_cosmics'][i] > 3000000:
                            if (pd_hdf['elapsed_time'][i]) > 900:
                                list_of_dl1_files.append(dl1_file)
                                optical_eff[int(pd_hdf['runnumber'][i])] = (pd_hdf['mu_effi_mean'][i])
                                mu_width[int(pd_hdf['runnumber'][i])] = (pd_hdf['mu_width_mean'][i])
                                mu_number.append(pd_hdf['num_contained_mu_rings'][i])


In [ ]:
for file in list_of_dl1_files:
    pd_hdf = pd.read_hdf(file, 'runsummary/table')
    #pd_hdf_cosmic = pd.read_hdf(dl1_file, 'cosmics/table')
    for key,time in enumerate(pd_hdf['time']):
        dt_object = datetime.utcfromtimestamp(time)
        formatted_time = dt_object.strftime('%Y-%m-%d %H:%M:%S')
        print(f"for run {pd_hdf['runnumber'][key]} the date is {formatted_time}\n")

In [ ]:
for file in list_of_dl1_files:
    pd_hdf = pd.read_hdf(file, 'runsummary/table')
    print(f"treating {file}")
    for i,item in enumerate(pd_hdf['mean_altitude']):
        if (np.rad2deg(abs(pd_hdf['min_altitude'][i] - pd_hdf['max_altitude'][i])) <= 1) and \
            (np.rad2deg(abs(pd_hdf['min_azimuth'][i] - pd_hdf['max_azimuth'][i])) <= 1):
                print(f"for run {pd_hdf['runnumber'][i]} we have altitude = {np.rad2deg(item)} and azimuth = {np.rad2deg(pd_hdf['mean_azimuth'][i])} and optical efficiency {pd_hdf['mu_effi_mean'][i]}")

In [ ]:
set(list_of_dl1_files)

In [ ]:
sorted_dict = {key: optical_eff[key] for key in sorted(optical_eff)}
runs = [key for key in sorted_dict.keys()]
effs = [sorted_dict[key] for key in sorted_dict.keys()]
plt.scatter(runs, effs, marker = 'x', s = 25, alpha = 0.7)
plt.grid(alpha = 0.5)
plt.xlabel('Run Number')
plt.ylabel('Optical Efficiency')

In [ ]:
plt.figure(figsize = (12,9))
sns.regplot(x = runs, y = effs, x_bins=70, color = 'k', label = 'Real Data')
plt.grid(alpha = 0.5)
plt.title("Optical Efficiency")

In [ ]:
plt.figure(figsize = (12,9))
sns.regplot(x = runs, y = effs, x_bins=70, color = 'k', label = 'Real Data')
plt.grid(alpha = 0.5)
plt.title("Optical Efficiency")

In [ ]:
sorted_dict = {key: mu_width[key] for key in sorted(mu_width)}
runs = [key for key in sorted_dict.keys()]
mu_width = [sorted_dict[key] for key in sorted_dict.keys()]
plt.scatter(runs, mu_width, marker = 'x', s = 25, alpha = 0.7)
plt.grid(alpha = 0.5)

In [ ]:
plt.figure(figsize = (12,9))
sns.regplot(x = runs, y = mu_width, x_bins=70, color = 'darkorange', label = 'Real Data')
plt.grid(alpha = 0.5)
plt.title("Muon Width")

In [ ]:
sorted_dict = {key: date[key] for key in sorted(date)}
runs = [key for key in sorted_dict.keys()]
new_date = [sorted_dict[key] for key in sorted_dict.keys()]
plt.scatter(runs, new_date, marker = 'x', s = 25, alpha = 0.7)
plt.grid(alpha = 0.5)

In [ ]:
sorted_dict = {key: num_cosmic[key] for key in sorted(num_cosmic)}
runs = [key for key in sorted_dict.keys()]
new_num_cosmic = [sorted_dict[key] for key in sorted_dict.keys()]
plt.scatter(runs, new_num_cosmic, marker = 'x', s = 25, alpha = 0.7)
plt.grid(alpha = 0.5)

In [ ]:
sorted_cosmic = {key: date[key] for key in sorted(date)}
runs = [key for key in sorted_cosmic.keys()]
sorted_elapsed = {key: elapsed_time[key] for key in sorted(elapsed_time)}
runs_elapsed = [key for key in sorted_elapsed.keys()]
new_cosmic = np.array([sorted_cosmic[key] for key in sorted_cosmic.keys()])
new_elapsed = np.array([sorted_elapsed[key] for key in sorted_elapsed.keys()])
cosmic_rate = new_cosmic/new_elapsed
plt.scatter(runs, cosmic_rate, marker = 'x', s = 25, alpha = 0.7)
plt.grid(alpha = 0.5)

In [ ]:
cosmic_rate = []
x_runs = []
for key,item in num_cosmic.items():
    x_runs.append(key)
    cosmic_rate.append(item/elapsed_time[key])
    
plt.scatter(x_runs, cosmic_rate)
    

In [ ]:
for time in elapsed_time:
    print(time/60)

In [ ]:
from astropy.io import fits

# Path to your FITS file
fits_file_path = '/Users/vdk/muons2024/muons_LST-1.Run16869.0095.fits'


with fits.open(fits_file_path) as hdul:
    # hdul is a list-like collection of HDU (Header/Data Unit) objects.
    
    # Print the contents of the FITS file
    hdul.info()
    
    # Access the primary HDU (HDU 0)
    primary_hdu = hdul[1]
    
    # Access data and header of the primary HDU
    data = primary_hdu.data
    header = primary_hdu.header
    
    # Print header information
    print(header)
    


In [ ]:
keys = [
    ('event_id', '>i8'), 
    ('event_time', '>f8'), 
    ('mc_energy', '>i8'), 
    ('ring_size', '>f4'), 
    ('size_outside', '>f4'), 
    ('ring_center_x', '>f8'), 
    ('ring_center_y', '>f8'), 
    ('ring_radius', '>f8'), 
    ('ring_width', '>f8'), 
    ('good_ring', 'i1'), 
    ('muon_efficiency', '>f8'), 
    ('ring_containment', '>f8'), 
    ('ring_completeness', '>f8'), 
    ('impact_parameter', '>f8'), 
    ('impact_x_array', '>f8'),
    ('impact_y_array', '>f8'), 
    ('radial_stdev', '>f8'), 
    ('radial_skewness', '>f8'), 
    ('radial_excess_kurtosis', '>f8'), 
    ('num_pixels_in_ring', '>i8'), 
    ('mean_pixel_charge_around_ring', '>f8'), 
    ('hg_peak_sample', '>i8'), 
    ('lg_peak_sample', '>i8')
]

for i, item in enumerate(data[0]):
    print(f"{keys[i][0]} = {item}")

In [ ]:

EventSource